In [34]:
# !pip install -U git+https://github.com/albumentations-team/albumentations
# import os
# os.system(' cp "drive/My Drive/train.zip" "train.zip" ')
# os.system(' unzip train.zip' )

In [24]:
import numpy as np
import cv2
import os
import pandas as pd
import string
import matplotlib.pyplot as plt
import tqdm
from os.path import join
from collections import Counter

import editdistance

import keras

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional, Dropout, AveragePooling2D, Input, ReLU
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.callbacks import Callback
from sklearn.model_selection import GroupKFold

import imutils
from keras.optimizers import *

import albumentations as A
import pickle
import gc

In [3]:
print('TensorFlow version:', tf.__version__)
print('Keras version:', keras.__version__)

TensorFlow version: 2.3.0
Keras version: 2.4.3


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#В этой папке лежат txt файлы перевода
trans_dir = os.path.join('train', 'words')
#В этой папке лежат  jpg файлы изображений
image_dir = os.path.join('train', 'images')

print(len(os.listdir(trans_dir)))

6196


In [6]:
max_len = 60

In [7]:
def text_to_labels(text, letters):
    return list(map(lambda x: letters.index(x), text))

english_letters = ''

def read_target(trans_dir, english_drop = True):
    target_folders = [os.path.join(trans_dir, p) for i, p in enumerate( sorted(os.listdir(trans_dir)))]
    len_target = []
    real_targets = []
    for file in target_folders:
        with open(file, 'r',  encoding='utf8') as file:
            data = file.read()
                       
            if english_drop:
              data = data.replace('u', '')
              data = data.replace('–', ' ')
              data = data.replace('l', '|')
              data = data.replace('a', 'а').replace('b', 'б').replace('k', '')
              data = data.replace('ǂ', '')
            data = ' '.join(data.split())
            data = data.lstrip().rstrip()
        if len(set(data) & set(english_letters)) != 0 and english_drop:
          real_targets += ['12345']
          len_target += [5]
        else:
          real_targets += [data]
          len_target += [len(data)]        
        
    max_label_len = max(len_target)
    letters = sorted(list(set(''.join(real_targets))))

    convert_targets = [text_to_labels(t, letters) for t in real_targets]
    
    targets = pad_sequences(convert_targets, 
                             maxlen=max_label_len, 
                             padding='post',
                             value=len(letters))
    return np.array(targets), real_targets, letters, np.array(len_target)

# os.system(' cp "drive/My Drive/checker.py" "checker.py" ')
# !python checker.py 'train/words'

In [8]:
targets, real_targets, letters, len_targets = read_target(trans_dir)
targets_f, real_targets_f, letters_f, len_targets_f = read_target(trans_dir, False)

In [9]:
print(letters)

[' ', ')', '+', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '[', ']', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'm', 'n', 'o', 'p', 'r', 's', 't', '|', '×', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'і', 'ѣ', '…', '⊕', '⊗']


In [13]:
def block(x, f):
    def b(x):
        conv_1 = Conv2D(f, (3, 3),  padding='same')(x)
        bn = BatchNormalization()(conv_1)
        rel = ReLU()(bn)
        return rel
    return b

def block_0(x, f):
    def b(x):
        conv_1 = Conv2D(f, (3, 3), activation='relu',  padding='same')(x)
        bn = BatchNormalization()(conv_1)
        return bn
    return b
    
max_label_len = max(len_targets)
max_label_len = 60

def crnn(w_good = 64, h_good = 64 * 8):

    inputs = Input(shape=(w_good, h_good, 1))

    x = block(inputs, 64)(inputs)
    x = block(x, 64)(x)
    x = block(x, 64)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=2)(x)
    x = Dropout(rate=0.2)(x)

    x = block(x, 128)(x)
    x = block(x, 128)(x)
    x = block(x, 128)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)
    x = Dropout(rate=0.2)(x)

    x = block(x, 256)(x)
    x = block(x, 256)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 1))(x)
    x = Dropout(rate=0.3)(x)

    x = block_0(x, 512)(x)
    x = block_0(x, 512)(x)
    x = MaxPool2D(pool_size=(2, 2), strides=(2, 1))(x)
    x = Dropout(rate=0.3)(x)

    x = block_0(x, 768)(x)
    x = block_0(x, 768)(x)
    ap = AveragePooling2D(pool_size=(4, 1), padding='same' )(x)

    squeezed = Lambda(lambda x: K.squeeze(x, 1))(ap)

    blstm_1 = Bidirectional(LSTM(160, return_sequences=True, dropout=0.5))(squeezed)
    blstm_2 = Bidirectional(LSTM(160, return_sequences=True, dropout=0.5))(blstm_1)
    blstm_3 = Bidirectional(LSTM(196, return_sequences=True, dropout=0.5))(blstm_2)

    outputs = Dense(len(letters) + 1, activation='softmax')(blstm_3)
    act_model = Model(inputs=[inputs], outputs=outputs)

    the_labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')

    def ctc_lambda_func(args):
        y_pred, labels, input_length, label_length = args

        return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, the_labels, input_length, label_length])

    model = Model(inputs=[inputs, the_labels, input_length, label_length], outputs=loss_out)

    return model, act_model


model, act_model = crnn()
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 512, 1)] 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 512, 64)  640         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 512, 64)  256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 64, 512, 64)  0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

In [14]:
output_feats = model.layers[-5].output_shape[1]
output_feats

126

In [15]:
import random 


def generate_batches(ind, targets, len_targets, batch_size = 8, root_images=None, root_words=None,
                     w_good = 128, h_good = 1024, aug = True):
    counter = 0

    ind = np.sort(ind)

    name_files_words  = [os.path.join(root_words, p) for i, p in enumerate( sorted(os.listdir(root_words)))]
    name_files_images = []
    for i in ind:
      name_files_images += [name_files_words[i].replace(root_words, root_images).replace('txt', 'jpg')]
    
    k = 0
    while True:
      random.seed(k)
      new_ind = np.arange(len(ind))
      random.shuffle(new_ind)
      targets_shuffle = targets[ind[new_ind]]
      len_targets_shuffle = len_targets[ind[new_ind]]
      name_files_images_shuffle = [name_files_images[x] for x in new_ind]
      k += 1

      num_iter = len(name_files_images) // batch_size + int((len(name_files_images) % batch_size) > 0)
      for counter in range(0, num_iter):
          inames = name_files_images_shuffle[counter * batch_size : (counter + 1) * batch_size]
          list_img = []
          dim_img = []
          for path in inames:
              img = image_array_new[dict_name[path]]
              if aug:
                p = 0.35
                t =  A.Compose([
                    A.Rotate([-3, 3], p = p, border_mode=1),
                ])

                aug_num = random.choice(np.arange(1, 15))
                if aug_num in [1,2,3,4,5,6]:
                    ind_num = random.choice(np.arange(30))
                    img = image_array_new_aug0[dict_name[path]][ind_num]
                elif aug_num in [7,8,9]:
                    ind_num = random.choice(np.arange(15))
                    img = image_array_new_aug1[dict_name[path]][ind_num]
                elif aug_num in [10,11]:
                    ind_num = random.choice(np.arange(10))
                    img = image_array_new_aug2[dict_name[path]][ind_num]
                elif aug_num in [12]:
                    ind_num = random.choice(np.arange(5))
                    img = image_array_new_aug3[dict_name[path]][ind_num]
                else:
                    ind_num = random.choice(np.arange(10))
                    img = image_array_new_aug4[dict_name[path]][ind_num]                 
                img = t( image = img)['image']

              else:
                1

              list_img += [img]
              

          list_img = normalization(list_img)
          target = targets_shuffle[counter * batch_size : (counter + 1) * batch_size] 
          len_input = np.array(([output_feats] * len(target)))
          len_target = len_targets_shuffle[counter * batch_size : (counter + 1) * batch_size]
          dim_img = np.array(dim_img)

          yield ( [list_img,  target, len_input, len_target], np.zeros(len(target)) )

In [16]:
def normalization(imgs):
    """Normalize list of images"""

    imgs = np.asarray(imgs).astype(np.float32)
    # print(imgs.shape)
    _, h, w, _ = imgs.shape

    for i in range(len(imgs)):
        m, s = cv2.meanStdDev(imgs[i])
        imgs[i] = imgs[i] - m[0][0]
        imgs[i] = imgs[i] / s[0][0] if s[0][0] > 0 else imgs[i]

    return imgs

In [17]:
def process_image(img, w_good=128, h_good=1024, fill = 190):
    w, h = img.shape

    if w < 200 and h / w < 2.5:
      new_w = int(w_good * 0.8)
    else:
      new_w = w_good
    new_h = int(h * (new_w / w))
    if new_h > img.shape[1] :
        img = cv2.resize(img, (new_h, new_w), interpolation = cv2.INTER_CUBIC )
    else:
        img = cv2.resize(img, (new_h, new_w), interpolation = cv2.INTER_AREA )
    # img = (img - np.mean(img)) / ( np.std(img) + 1e-8 )

    w, h = img.shape

    img = img.astype('float32')

    if w < w_good:
        add_zeros = np.full((w_good - w, h), fill)
        img = np.concatenate((img, add_zeros))
        w, h = img.shape

    if h < h_good:
        add_zeros = np.full((w, h_good - h), fill)
        img = np.concatenate((img, add_zeros), axis=1)
        w, h = img.shape

    if h > h_good or w > w_good:
        dim = (h_good, w_good)
        if h_good > img.shape[1] :
          img = cv2.resize(img, dim, interpolation = cv2.INTER_CUBIC )
        else:
          img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA )

    # print(img)
    img = img.astype(np.float32)

    img = img.reshape((img.shape[0], img.shape[1], 1))

    return img


In [18]:
w_good = 64
h_good = 64 * 8


name_files_words  = [os.path.join(trans_dir, p) for i, p in enumerate( sorted(os.listdir(trans_dir)))]
name_files_images = [p.replace(trans_dir, image_dir).replace('txt', 'jpg') for p in  name_files_words]
def make_dict_photo():
  image_array = []
  img_sizes = []
  i = 0
  for p in tqdm.tqdm_notebook(name_files_images):
    img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
    med = np.clip(np.median(img[img <= 250]), 180, 200)
    img[img > 250] = med

    if img.shape[0] > 1000:
      if p == 'train/images/221_10_22.jpg' or p == 'train/images/221_10_23.jpg':
        img = imutils.rotate_bound(img, 90)
      else:
        img = imutils.rotate_bound(img, -90)
    elif img.shape[0] > img.shape[1] and img.shape[0] >= 200 and img.shape[0] / img.shape[1] > 2:
      img = imutils.rotate_bound(img, -90)


    image_array += [ process_image(img, w_good, h_good, med) ]
  image_array_new = np.array(image_array).astype('uint8')
  return image_array_new

image_array_new = make_dict_photo()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


Делаем аугментации, затем их надо сохранить в pickle чтобы каждый раз не повторять этот процесс

In [20]:
# os.system(' cp -avr "drive/My Drive/aug.py" "aug.py" ')
# import aug as augment

# def make_aug_photo(image_array_new, k_aug, numaug, dis = True):
#     image_array_new_aug = np.zeros([image_array_new.shape[0], numaug] +list(image_array_new.shape[1:]) , dtype = 'uint8')
#     for i in tqdm.tqdm_notebook(range(image_array_new.shape[0])):
#         img = np.squeeze(image_array_new[i])
#         list_img = np.zeros([ numaug] + list(image_array_new.shape[1:]) , dtype = 'uint8')
#         for k in range(numaug):
#           if dis:
#             list_img[k, :, :, 0] = augment.distort(img, k_aug)
#           else:
#             list_img[k, :, :, 0] = augment.stretch(img, k_aug)

#         image_array_new_aug[i] = list_img

#     return image_array_new_aug

# image_array_new_aug0 = make_aug_photo(image_array_new, 3, 30)
# image_array_new_aug1 = make_aug_photo(image_array_new, 4, 15)
# image_array_new_aug2 = make_aug_photo(image_array_new, 5, 10)
# image_array_new_aug3 = make_aug_photo(image_array_new, 6, 5)
# image_array_new_aug4 = make_aug_photo(image_array_new, 3, 10, False)

In [21]:
with open('drive/My Drive/distort_3_0.pickle', 'rb') as f:
    image_array_new_aug0 = pickle.load(f)

with open('drive/My Drive/distort4_0.pickle', 'rb') as f:
    image_array_new_aug1 = pickle.load(f)

with open('drive/My Drive/distort5_0.pickle', 'rb') as f:
    image_array_new_aug2 = pickle.load(f)

with open('drive/My Drive/distort6_0.pickle', 'rb') as f:
    image_array_new_aug3 = pickle.load(f)

with open('drive/My Drive/stretch3_0.pickle', 'rb') as f:
    image_array_new_aug4 = pickle.load(f)

In [22]:
gc.collect()
dict_name = {p:i for i,p in enumerate(name_files_images)}

In [27]:
names_pref = [n.split('/')[-1].split('_')[0] for n in name_files_words]

bad_ind = ['368_3_0', '460_9_9' ,'337_46_6']
bad_ind = [i for i, x in enumerate(name_files_words) if x.split('/')[-1].split('.')[0] in bad_ind]


gp = GroupKFold(5)

split_list = []
for train_ind, val_ind in gp.split(names_pref,names_pref, names_pref):
    new_train_ind = []
    for ind in np.arange(len(names_pref)):
    # for ind in train_ind:
      if real_targets[ind] != '12345' :
        if ind not in bad_ind :
            if len_targets_f[ind] < 63:
              new_train_ind += [ind]
    new_val_ind = []
    for ind in val_ind:
      if real_targets[ind] != '12345' :
        if ind not in bad_ind:
          new_val_ind += [ind]
    split_list += [(new_train_ind, new_val_ind, val_ind)]

In [28]:
class Show_lr(Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.lr
        # If you want to apply decay.
        decay = self.model.optimizer.decay
        
        iterations = self.model.optimizer.iterations
        lr_with_decay = lr / (1. + decay * K.cast(iterations, K.dtype(decay)))
        # print(epoch, lr, decay, iterations, K.cast(iterations, K.dtype(decay)))
        print(K.eval(lr_with_decay))

In [29]:
class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, valid_data, steps):
        """
        valid_data is a TFRecordDataset with batches of 100 elements per batch, shuffled and repeated infinitely. 
        steps defines the amount of batches per epoch
        """
        super(Metrics, self).__init__()
        self.valid_data = valid_data
        self.steps = steps

    def on_train_begin(self, logs={}):
        self.cer = []
        self.wer = []

    def on_epoch_end(self, epoch, logs={}):

        if epoch % 5 != 0:
          return

        labels = []
        prediction = []
        for i in range(self.steps) :
            data = next(self.valid_data)
            # print(data[0][0].shape)
            prediction += [act_model.predict([data[0][0]])]
            labels += [data[0][1]]
        prediction =  np.concatenate(prediction)
        labels =  np.concatenate(labels)

        decoded = K.ctc_decode(prediction, 
                              input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                              greedy=True)[0][0]
        out = K.get_value(decoded)

        numCharErr = 0
        numCharTotal = 0
        numStringOK = 0
        numStringTotal = 0

        word_eds, word_true_lens = [], []

        real_answer = []
        for x in labels:
            real_answer += [ ''.join([letters[i] for i in x if i != len(letters) ])]
        for i in range(len(out)):
            pred = ''
            for p in out[i]:
                if int(p) != -1:
                    pred+=letters[int(p)]

                    
            true = real_answer[i]
            
            numStringOK += 1 if true == pred else 0
            numStringTotal += 1
            dist = editdistance.eval(pred, true)
            numCharErr += dist
            numCharTotal += len(true)
            
            pred_words = pred.split()
            true_words = true.split()
            word_eds.append(editdistance.eval(pred_words, true_words))
            word_true_lens.append(len(true_words))

        charErrorRate = numCharErr / numCharTotal
        wordErrorRate = sum(word_eds) / sum(word_true_lens) 
        stringAccuracy = numStringOK / numStringTotal
        print('Character error rate: %f%%. Word error rate: %f%%. String accuracy: %f%%.' % \
              (charErrorRate*100.0,wordErrorRate*100.0, stringAccuracy*100.0))

        print('-------------------------------')

In [30]:
def scheduler(epoch, lr):
   lr = 0.001
   if epoch < 100:
     return lr
   else:
     return lr * 0.965 ** ((epoch - 100) + 1) 
callback_lr_my = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [31]:
# tf.keras.backend.clear_session()
# del model
# del act_model

In [33]:
num_e = 3

for train_index, test_index, real_test_index in split_list[3:]:
    num_e += 1
    print(len(train_index), len(test_index), len(real_test_index) )
    batch_size = 32
    epochs = 300

    gen_tr = generate_batches(train_index, targets, len_targets, batch_size, image_dir, trans_dir,w_good, h_good)
    gen_val = generate_batches(test_index, targets, len_targets, batch_size, image_dir, trans_dir, w_good, h_good, False)
    # hui
    
    os.makedirs('drive/My Drive/checkpoint{}'.format(num_e), exist_ok=True)

    filepath="drive/My Drive/checkpoint{}".format(num_e) + "/model_{epoch}.hdf5"
    es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=500)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.7, verbose = 1,
                                  patience=5, min_lr=0.00001)
    print_lr = Show_lr() 
    checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

    model, act_model = crnn(w_good, h_good)


    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = Adam(0.001), metrics=['accuracy'])

    steps_per_epoch =  len(train_index) // batch_size + int((len(train_index) % batch_size) > 0)  
    validation_steps = len(test_index) // batch_size + int((len(test_index) % batch_size) > 0)


    # gen_val_m = generate_batches(test_index, targets, len_targets, batch_size, image_dir, trans_dir, w_good, h_good, False)
    # validation_steps_m = len(test_index) // batch_size + int((len(test_index) % batch_size) > 0)
    # metric = Metrics(gen_val_m, validation_steps_m)

    # callbacks_list = [checkpoint, es, print_lr, callback_lr_my, metric]

    # history = model.fit(gen_tr, steps_per_epoch = steps_per_epoch, epochs=epochs, verbose=1,
    #                 callbacks=callbacks_list,  
    #                 validation_data = gen_val, validation_steps = validation_steps)
  

    callbacks_list = [print_lr, callback_lr_my]
    history = model.fit(gen_tr, steps_per_epoch = steps_per_epoch, epochs=epochs, verbose=1,  callbacks=callbacks_list)
    
    filepath = os.path.join('drive/My Drive/checkpoint{}'.format(num_e), 'model45.hdf5')
    model.save_weights(filepath)
    
    hi
    tf.keras.backend.clear_session()
    del model
    del act_model